Сначала импортируем классы и инциализируем их (в апи это происходит в dependencies)

In [1]:
import os
import geopandas as gpd
import pandas as pd

os.environ["APP_ENV"] = 'development'

from iduconfig import Config

from app.logic.logger_setup import setup_logger

from app.api.urbandb_api_gateway import UrbanDBAPI
from app.api.genbuilder_gateway import GenbuilderInferenceAPI

from app.logic.postprocessing.generation_params import GenParams, ParamsProvider
from app.logic.building_generation.building_capacity_optimizer import CapacityOptimizer
from app.logic.building_generation.maximum_inscribed_rectangle import MIR
from app.logic.building_generation.segments import SegmentsAllocator
from app.logic.building_generation.plots import PlotsGenerator
from app.logic.building_generation.buildings import ResidentialBuildingsGenerator
from app.logic.building_generation.residential_generator import ResidentialGenBuilder
from app.logic.building_generation.residential_service_generation import ResidentialServiceGenerator
from app.logic.building_generation.building_params import (
    BuildingGenParams,
    BuildingParamsProvider,
    PARAMS_BY_TYPE
)

config = Config()
setup_logger(config, log_level="INFO")

urban_db_api = UrbanDBAPI(config)

generation_parameters = GenParams()
params_provider = ParamsProvider(generation_parameters)

buildings_params = BuildingGenParams(params_by_type=PARAMS_BY_TYPE)
buildings_params_provider = BuildingParamsProvider(base=buildings_params)

Далее используем уже инициализированные классы (из logic.building_generator.residential_generator)

In [2]:
building_capacity_optimizer = CapacityOptimizer(buildings_params_provider)
max_rectangle_finder = MIR()
segments_allocator = SegmentsAllocator(building_capacity_optimizer, buildings_params_provider)
plots_generator = PlotsGenerator(params_provider, buildings_params_provider)
residential_buildings_generator = ResidentialBuildingsGenerator()
residential_generator = ResidentialGenBuilder(building_capacity_optimizer, max_rectangle_finder, 
                    segments_allocator, plots_generator, residential_buildings_generator, params_provider)
residential_service_generator = ResidentialServiceGenerator(params_provider)

Здесь происходит имитиация передачи входных параметров - полигонов кварталов, целевой жилой площади по всему проекту, типу этажности и целевому сценарию плотности.

Типы этажности (задаются в building_params):
- private: ИЖС
- low: 2-4 этажа МКД
- medium: 5-8 этажей МКД
- high: 9-16 этажей МКД
- extreme: 16+ этажей МКД

Сценарии плотности рассчитываются по FAR (суммарная площадь всех этажей здания / площадь участка) для каждого участка.

Сценарии (логика выбора в building_capacity_optimizer.pick_indices):
- min: максимальная площадь участка, минимальная здания
- mean: средние значения из допустимых
- max: минимальная площадь участка, максимальная здания

In [3]:
blocks = gpd.read_file('zones.geojson')
residential_blocks = blocks[blocks['zone'] == 'residential']
residential_la_target = 10000
default_floors_group = 'medium'
density_scenario = 'min'

Целевая жилая площадь распределяется по кварталам соразмерно их площади, категория этажности может быть задана отдельно в параметрах входных кварталов, если нет - используется дефолтная.

In [4]:
residential_blocks["la_target"] = (
    residential_la_target
    * residential_blocks.geometry.area
    / residential_blocks.geometry.area.sum()
)
residential_blocks["floors_group"] = residential_blocks.get("floors_group")
residential_blocks["floors_group"] = residential_blocks["floors_group"].fillna(
    default_floors_group
)
residential_blocks.head(3)

,zone_id,zone,floors_group,la_target,geometry
0,1,residential,medium,2385.906444,"POLYGON ((359002.284 6648270.471, 359136.097 6..."
1,2,residential,medium,4916.048384,"POLYGON ((358808.367 6648679.85, 358993.211 66..."
2,3,residential,private,2698.045172,"POLYGON ((359332.282 6648424.696, 359502.384 6..."


В классе CapacityOptimizer предварительно рассчитывается то, сколько зданий требуется уместить в квартал, и сколько возможно.

В начале используются параметры зданий и участков, соответствующие исходному сценарию плотности и типу этажности.

building_need: la_target / living_area

building_capacity: plot_side / block perimeter

Далее, пока building_need < building_capacity, параметры изменяются в сторону увеличения плотности с приоритетом расположения участков по длинной стороне квартала.

Итоговые параметры зданий и участков сохраняются в атрибуты кварталов.

In [5]:
residential_blocks = building_capacity_optimizer.compute_blocks_for_gdf(
    residential_blocks, density_scenario
)
residential_blocks.head(3)

,zone_id,zone,floors_group,la_target,geometry,building_need,building_capacity,buildings_count,plot_front,plot_depth,...,building_length,building_width,floors_count,living_per_building,total_living_area,la_diff,la_ratio,far_initial,far_final,far_diff
0,1,residential,medium,2385.906444,"POLYGON ((359002.284 6648270.471, 359136.097 6...",2.0,NaN,2.0,120.0,50.0,...,40.0,12.0,5.0,1200.0,2400.0,14.093556,1.005907,0.400000,NaN,NaN
1,2,residential,medium,4916.048384,"POLYGON ((358808.367 6648679.85, 358993.211 66...",5.0,NaN,5.0,120.0,50.0,...,40.0,12.0,5.0,1200.0,6000.0,1083.951616,1.220492,0.400000,NaN,NaN
2,3,residential,private,2698.045172,"POLYGON ((359332.282 6648424.696, 359502.384 6...",65.0,NaN,65.0,40.0,22.5,...,8.0,8.0,1.0,41.6,2704.0,5.954828,1.002207,0.071111,NaN,NaN


In [6]:
residential_blocks.explore()

В max_rectangle_finder происходит процесс вписывания прямоугольников в полигоны кварталов с целью закрыть большую часть квартала ими. У прямоугольников есть нижний порог по стороне. Этот шаг нужен для нормализации застройки до сетчатой и имитации серий строительства.

Пока что наиболее затратный по времени этап.

In [7]:
segments = max_rectangle_finder.pack_inscribed_rectangles_for_gdf(
    residential_blocks,
    step= generation_parameters.rectangle_finder_step,
    min_side= generation_parameters.minimal_rectangle_side,
    n_jobs= generation_parameters.jobs_number,
)
segments.head(3)

Packing rectangles (parallel):   0%|          | 0/3 [00:00<?, ?it/s]

,geometry,width,height,area,angle,src_index,rect_id
0,"POLYGON ((359317.117 6648412.513, 359003.165 6...",320.0,345.0,110400.0,114.493937,0,0
1,"POLYGON ((359450.824 6648119.036, 359273.373 6...",65.0,195.0,12675.0,114.493937,0,1
2,"POLYGON ((359458.334 6648048.282, 359385.534 6...",40.0,80.0,3200.0,114.493937,0,2


Далее building_need и building_capacity пересчитывается для сегментов.

In [8]:
(
    residential_blocks,
    segments,
) = segments_allocator.update_blocks_with_segments(
    residential_blocks,
    segments,
    far=density_scenario,
)
segments.head(3)

,geometry,width,height,area,angle,src_index,rect_id,plots_capacity,plot_front,plot_depth,la_target,far_initial,building_length,building_width,floors_count,floors_group
0,"POLYGON ((359317.117 6648412.513, 359003.165 6...",320.0,345.0,110400.0,114.493937,0,0,2,120.0,50.0,2385.906444,0.4,40.0,12.0,5.0,medium
1,"POLYGON ((359450.824 6648119.036, 359273.373 6...",65.0,195.0,12675.0,114.493937,0,1,1,120.0,50.0,2385.906444,0.4,40.0,12.0,5.0,medium
2,"POLYGON ((359458.334 6648048.282, 359385.534 6...",40.0,80.0,3200.0,114.493937,0,2,0,120.0,NaN,2385.906444,0.4,40.0,12.0,5.0,medium


In [9]:
segments.explore()

В сегменты вписываются кварталы: от краев сегментов строятся 4 полосы глубиной, равной глубине участка plot_depth. Затем квадраты пересечений на углах сливаются по длинной стороне сегментов. Эти полосы делятся по ширине участков plot_side. Если в процессе образуются слишком узкие участки, они сливаются с соседними.

После создания участков параметры зданий пересчитываются для соответствия таргету по жилой площади и плотности.

In [10]:
plots = plots_generator.generate_plots(segments)
plots.head(3)

,width,height,area,angle,src_index,rect_id,plots_capacity,plot_front,plot_depth,la_target,...,segment_side,geometry,plot_area,living_per_building,living_area,total_living_area,la_diff,la_ratio,far_final,far_diff
2,415.0,265.0,109975.0,110.458382,2,0,10,40.0,22.5,2698.045172,...,corner_br,"POLYGON ((359643.469 6648459.097, 359622.388 6...",506.25,41.6,41.6,2694.9,-3.145172,0.998834,0.058355,-0.012756
3,70.0,110.0,7700.0,110.458382,2,2,2,40.0,22.5,2698.045172,...,corner_br,"POLYGON ((359796.423 6648070.552, 359775.342 6...",506.25,41.6,41.6,2694.9,-3.145172,0.998834,0.058355,-0.012756
4,70.0,110.0,7700.0,110.458382,2,2,2,40.0,22.5,2698.045172,...,corner_tr,"POLYGON ((359693.361 6648032.104, 359685.497 6...",506.25,41.6,41.6,2694.9,-3.145172,0.998834,0.058355,-0.012756


In [11]:
plots.explore()

В участки вписываются полигоны зданий с итоговыми параметрами длины, ширины и высоты, начиная от середины квартала. Гарантируется отступ в 3 метра от границ участка.

In [12]:
buildings_gdf = residential_buildings_generator.generate_buildings_from_plots(plots)
buildings_gdf.head(3)

,geometry,building_length,building_width,floors_count,living_area,src_index
0,"POLYGON ((359634.142 6648463.356, 359631.346 6...",8.0,8.0,1.0,41.6,2
1,"POLYGON ((359787.096 6648074.811, 359784.3 664...",8.0,8.0,1.0,41.6,2
2,"POLYGON ((359705.115 6648044.227, 359702.319 6...",8.0,8.0,1.0,41.6,2


In [13]:
buildings_gdf.explore()

Далее мы получаем нормативы по сервисам вида Х мест на 1000 человек из urban_db. Они уникальны для каждой территории. 

На основе этих нормативов и суммы распределенной жилой площади для каждого квартала определяются лимиты для сервисов (требуемая вместимость). В пространстве кварталов, не занятых участками жилых зданий, алгоритм старается разместить участки сервисов (параметры участков и зданий сервисов находятся в service_projects.geojson), пока весь лимит вместимости не будет покрыт, либо пока не закончится место. Для каждого сервиса есть несколько проектов разной вместимости и площади участков, алгоритм отдает приоритет закрытию лимитов большими зданиями.

In [ ]:
territory_id = 1
service_normatives = await urban_db_api.get_normatives_for_territory(territory_id)
residential_services = residential_service_generator.generate_services(residential_blocks, plots, 
                                                                buildings_gdf, service_normatives, 32636)

In [15]:
residential_services.head(3)

,src_index,service,project_id,capacity,floors_count,osm_url,address,geometry
0,0,Поликлиника,polyclinic_kazan_20,400.0,5,https://www.openstreetmap.org/relation/18102614,"23, улица Академика Сахарова, Азино-1, Советск...","POLYGON ((359195.887 6648205.977, 359213.927 6..."
1,0,Детский сад,kindergarten_140,140.0,2,https://www.openstreetmap.org/way/28202955,"11 к2, улица Коллонтай, Klochki, Невский округ...","POLYGON ((359135.037 6648236.582, 359110.29 66..."
2,1,Поликлиника,polyclinic_kazan_20,400.0,5,https://www.openstreetmap.org/relation/18102614,"23, улица Академика Сахарова, Азино-1, Советск...","POLYGON ((359340.493 6648671.509, 359363.387 6..."


In [16]:
residential_services.explore()

Далее жилые здания и здания сервисов объединяются в общий gdf и передаются на фронт.

In [17]:
result  = pd.concat([buildings_gdf, residential_services])
result.explore()